In [11]:
# simple_fuse: simple fusion from acoustic and txt
# run in MPC as : exec(open('simple_fuse_dev.py').read())
# 2019-12-20: initial work, it works!
# 2019-12-21: update to use silence feature from speech network
#             new data splitting (6000/2000/2039)
# 2019-12-24: Moodified from MSP-I+N data, LOSO

import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from calc_scores import calc_scores
from sklearn.linear_model import Ridge,SGDRegressor

ser = np.load('../MSPIN-SPEECH/5744npy/speech_5744-npy1.npy')
ter = np.load('../MSPIN-TEXT/5744npy-glove/text_5744-glove-npy7.npy')
st = np.load('../MSPIN-ST/5744npy-glove/st_5744-npy1.npy')

# split dev and test
split = 1148
ser_dev = ser[:split]
ser_test = ser[split:]
ter_dev = ter[:split]
ter_test = ter[split:]
st_dev = st[:split]
st_test = st[split:]

# load label 
vad = np.load('../improv_loso/data/vad.npy')

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) 
    scaled_vad = scaler.transform(vad) 
    vad = scaled_vad 
else:
    vad = vad

y_dev = vad[4596:5744]
y_test = vad[5744:] 

# SVR model
#svr_rbf = SVR(kernel='rbf', C=200, gamma=0.1, epsilon=0.01)
svr_rbf = SGDRegressor(loss="epsilon_insensitive")
# predicting valence
valence_model = svr_rbf.fit(np.array([ser_dev[:,0], ter_dev[:,0],st_dev[:,0]]).T, y_dev[:,0])
valence_pred = valence_model.predict(np.array([ser_test[:,0], ter_test[:,0],st_test[:,0]]).T)

ccc_v, pcc_v, rmse_v = calc_scores(valence_pred, y_test[:,0])

# predicting arousal
arousal_model = svr_rbf.fit(np.array([ser_dev[:,1], ter_dev[:,1],st_dev[:,1]]).T, y_dev[:,1])
arousal_pred = arousal_model.predict(np.array([ser_test[:,1], ter_test[:,1],st_test[:,1]]).T)

ccc_a, pcc_a, rmse_a = calc_scores(arousal_pred, y_test[:,1])

# predicting dominance
dominance_model = svr_rbf.fit(np.array([ser_dev[:,2], ter_dev[:,2],st_dev[:,2]]).T, y_dev[:,2])
dominance_pred = dominance_model.predict(np.array([ser_test[:,2], ter_test[:,2],st_test[:,2]]).T)

ccc_d, pcc_d, rmse_d = calc_scores(dominance_pred, y_test[:,2])



# vad_pred = np.vstack((valence_pred, arousal_pred, dominance_pred)).T
# filename = 'svm_mspin_loso/' + speech + '_' + text + '.npy' 
# np.save(filename, vad_pred)


In [10]:
#ST-5744
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.3139824112468437 0.5826243920050159 0.4296799533465514
PCC 0.3632431356863026 0.6103477965541306 0.47023249317485627
RMSE 0.454576494848281 0.3047701897662921 0.24807457123367666


In [12]:
#ST_st-5744
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.3314551287613665 0.5930932974974177 0.4449743682340698
PCC 0.38938517027782527 0.628383979291787 0.49120840008940897
RMSE 0.448233951477792 0.2966510441843596 0.24419691728254092
